In [ ]:
%cd /Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt

In [ ]:
# Varialble definations
path = '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/'
block_h, block_w = (120,120)

In [ ]:
import os
import math
import tensorflow
from keras.applications.vgg19 import VGG19


from keras.applications.densenet import DenseNet201
from keras.applications.densenet import DenseNet121

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, GlobalMaxPooling2D, Concatenate, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing import image


class ImageModel:
    def __init__(self, base_route, model_name, model_route="model.h5",
                 train_folder="train", validation_folder="val", epochs=10,
                 fine_tune: bool = False, fine_tune_epochs=5):

        self.model = None
        self.__base_model = None
        self.__train_directory_iterator = None
        self.__validation_directory_iterator = None

        self.__width = self.__height = 64
        self.__train_route = os.path.join(base_route, train_folder)
        self.__validation_route = os.path.join(base_route, validation_folder)
        self.__model_name = model_name
        self.__model_route = model_route

        self.__fine_tuning = fine_tune

        self.__epochs = epochs
        self.__batch_size = 64
        self.__fine_tune_epochs = fine_tune_epochs

        self.__early_stop = EarlyStopping(
            monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
        self.__checkpoint = self._get_model_checkpoint()

        self.train_size = 0
        self.validation_size = 0
        self.train_steps = 0
        self.validation_steps = 0

    def build(self):
        self.__train_directory_iterator = self._get_directory_iterator(
            self.__train_route, True)
        self.__validation_directory_iterator = self._get_directory_iterator(
            self.__validation_route)

        self.train_size = self.__train_directory_iterator.samples
        self.validation_size = self.__validation_directory_iterator.samples

        self._build_model(self.__train_directory_iterator.num_classes)

    def train(self):
        if self.__fine_tuning:
            self._set_fine_tune()
        else:
            self._set_transfer_learning()

        self.__model.fit_generator(
            self.__train_directory_iterator,
            steps_per_epoch=self.train_steps,
            epochs=self.__fine_tune_epochs,
            validation_data=self.__validation_directory_iterator,
            validation_steps=self.validation_steps,
            callbacks=[self.__checkpoint, self.__early_stop]
        )

        self.fit_all(train=self.__train_directory_iterator,
                     val=self.__validation_directory_iterator)

        self.__model.save(self.__model_route)

        metrics = self.__model.evaluate_generator(
            self.__validation_directory_iterator)

        return metrics

    def fit_all(self, train, val):
        for layer in self.__model.layers:
            layer.trainable = True

        self.__model.compile(optimizer=SGD(lr=0.01, momentum=0.6),
                             loss='categorical_crossentropy',
                             metrics=['accuracy'])

        self.__model.fit_generator(
            train,
            steps_per_epoch=self.train_steps,
            epochs=self.__epochs,
            validation_data=val,
            validation_steps=self.validation_steps,
            callbacks=[self.__checkpoint, self.__early_stop]
        )

    def _build_model(self, num_classes: int):

        if self.__model_name == "vgg19":
            self.__base_model = VGG19(weights='imagenet', include_top=False, input_shape=(
                self.__width, self.__height, 3))
        elif self.__model_name == "resnet":
            self.__base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(
                self.__width, self.__height, 3))
        elif self.__model_name == "densenet121":
            self.__base_model = DenseNet121(
                weights='imagenet', include_top=False, input_shape=(self.__width, self.__height, 3))
        elif self.__model_name == "DenseNet201":
            self.__base_model = DenseNet201(
                weights='imagenet', include_top=False, input_shape=(self.__width, self.__height, 3))

        x = self.__base_model.output

        x = Concatenate()(
            [GlobalAveragePooling2D()(x), GlobalMaxPooling2D()(x)])
        x = Dropout(0.2)(x)
        x = Dense(1024 / 2, activation='relu')(x)

        x = BatchNormalization()(x)
        x = Dropout(0.4)(x)
        predictions = Dense(num_classes, activation='softmax')(x)

        self.__model = Model(
            inputs=self.__base_model.input, outputs=predictions)

    def _set_transfer_learning(self):
        for layer in self.__base_model.layers:
            layer.trainable = False

        self.__model.compile(
            optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    def _set_fine_tune(self):
        layers_to_freeze = int(len(self.__base_model.layers) * 0.9)

        for layer in self.__model.layers[:layers_to_freeze]:
            layer.trainable = False
        for layer in self.__model.layers[layers_to_freeze:]:
            layer.trainable = True

        self.__model.compile(
            optimizer=SGD(lr=0.02, momentum=0.7),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

    def _get_model_checkpoint(self):
        return ModelCheckpoint(
            self.__model_route,
            monitor='val_acc',
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode='auto',
            period=1
        )

    def _get_directory_iterator(self, route, is_train: bool = False):
      image_generator = image.ImageDataGenerator(rescale=1.0 / 255, horizontal_flip=is_train,
                                                 vertical_flip=is_train)

      return image_generator.flow_from_directory(
          directory=route,
          target_size=(self.__width, self.__height),
          batch_size=self.batch_size,
          class_mode="categorical")

    @property
    def train_size(self):
        return self.__train_size

    @train_size.setter
    def train_size(self, train_size):
        self.__train_size = train_size
        self.train_steps = math.ceil(self.train_size / self.batch_size)

    @property
    def validation_size(self):
        return self.__validation_size

    @validation_size.setter
    def validation_size(self, validation_size):
        self.__validation_size = validation_size
        self.validation_steps = math.ceil(
            self.validation_size / self.batch_size)

    @property
    def batch_size(self):
        return self.__batch_size

    @batch_size.setter
    def batch_size(self, batch_size):
        self.__batch_size = batch_size
        self.train_steps = math.ceil(self.train_size / self.batch_size)
        self.validation_steps = math.ceil(
            self.validation_size / self.batch_size)

    @property
    def model(self):
        return self.__model

    @model.setter
    def model(self, model):
        self.__model = model


Trying feature extraction with VGG19 on one image folder

In [ ]:
vgg = ImageModel(base_route="tiny-imagenet-200",
                 epochs=20,
                 train_folder="train",
                 validation_folder="val/images",
                 fine_tune=True,
                 fine_tune_epochs=3,
                 model_name="vgg19",
                 model_route="vgg19_2.h5")
vgg.build()
vgg.model.summary()


In [1]:
import tensorflow as tf
model= tf.keras.applications.VGG19( include_top=True, weights="imagenet", input_tensor=None, input_shape=None, pooling=None, classes=1000, classifier_activation="softmax", )
print(model.summary())
model.save("vgg19.h5")

/Users/vitthal/miniforge3/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2022-12-05 11:59:17.804365: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-05 11:59:17.804643: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

574710816/574710816 [==============================] - 89s 0us/step
Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112